In [1]:
import pandas as pd
import numpy as np

In [2]:
rate_df = pd.read_csv('ratings.csv')
movies_all_info = pd.read_csv('movie_info.csv',index_col = 'movieId')

In [3]:
users = rate_df['userId'].values
movies = rate_df['movieId'].values
ratings = rate_df['rating'].values

mid_index = movies_all_info.index
mt_table = ['(no genres listed)', 'Action','Adventure','Animation','Children','Comedy','Crime','Documentary',
           'Drama','Fantasy', 'Film-Noir','Horror','IMAX','Musical','Mystery','Romance','Sci-Fi','Thriller',
           'War','Western']


movies_all_info_table = np.array([[0.0]*22]*150000)         # [isMinied,Avg,mt*20]
user_trend_att_table = np.array([[0]*20]*(max(users)+1))    # [mt_att*20] 
user_trend_tol_table = np.array([[0.0]*20]*(max(users)+1))  # [mt_tol*20] 

In [4]:
cnt = 0
for uid,mid,rating in zip(users,movies,ratings):
    
    if movies_all_info_table[mid][0] == 1:
        rating_avg = movies_all_info_table[mid][1]
        rating_diff = rating - rating_avg
        
        mts = movies_all_info_table[mid][2:]==1
        user_trend_att_table[uid][mts] += 1
        user_trend_tol_table[uid][mts] += rating_diff
        
    elif mid in mid_index:
        movies_all_info_table[mid][0] = 1
        movies_all_info_table[mid][1] = movies_all_info.loc[mid]['avg']
        movies_all_info_table[mid][2:] = movies_all_info.loc[mid][mt_table]
        
        rating_avg = movies_all_info_table[mid][1]
        rating_diff = rating - rating_avg
        
        mts = movies_all_info_table[mid][2:]==1
        user_trend_att_table[uid][mts] += 1
        user_trend_tol_table[uid][mts] += rating_diff
        
    else:
        continue
        
    cnt+=1

In [5]:


user_trend_df_cnt = pd.DataFrame(data=user_trend_att_table,columns=[mt+'_cnt' for mt in mt_table])
user_trend_df_rat = pd.DataFrame(data=user_trend_tol_table,columns=[mt+'_rat' for mt in mt_table])

user_trend_df = pd.concat([user_trend_df_cnt,user_trend_df_rat],axis=1)




In [6]:
mt_overall_voting = np.array([0]*len(mt_table))

user_trend_info_mtid = np.array([[0,0,0]]*len(user_trend_att_table))
user_trend_info_tol = np.array([[0.0,0.0,0.0]]*len(user_trend_att_table))

id_cnt = 0

for user_trend_array,tols in zip(user_trend_att_table,user_trend_tol_table):
    
    #mt_cnt_mean = np.mean(user_trend_array)
    #mt_cnt_std = np.std(user_trend_array)
    #mt_cnt_thres = mt_cnt_mean+mt_cnt_std
    #mt_attention = user_trend_array>mt_cnt_thres
    #mt_attention = [i for i,x in enumerate(user_trend_array>mt_cnt_thres) if x]
    mt_attention = sorted(range(len(user_trend_array)), key=lambda i: user_trend_array[i])[-3:]
    
    
    i = 0
    for mtid in mt_attention:
        
        mt_overall_voting += user_trend_array
        
        
        if user_trend_array[mtid] == 0:
            user_trend_array[mtid] = -1
        tol_avg = tols[mtid]/user_trend_array[mtid]
        
        user_trend_info_mtid[id_cnt][i] = mtid
        user_trend_info_tol[id_cnt][i] = tol_avg
        i += 1
        
        
        symbol = '+' if tol_avg>0 else ''
        
        if tol_avg > 1.0:
            comment = '非常寬容'
        elif tol_avg > 0.5:
            comment = '還算寬容'
        elif tol_avg > -0.5:
            comment = '評價平衡'
        elif tol_avg > -1.0:
            comment = '有點挑惕'
        else:
            comment = '非常挑惕'
            
        #print('user%06d 對%10s類電影%s(%s%f)' % (id_cnt,mt_table[mtid],comment,symbol,tol_avg))
    #print('')
    
    id_cnt += 1

In [7]:

import plotly
import plotly.graph_objs as go
import plotly.plotly as py

plotly.tools.set_credentials_file(username='nat99up', api_key='H6neWjamll2T6YtI5CG2')


fig = {
    'data' : [
      go.Histogram(
        histfunc = "sum",
        y = mt_overall_voting,
        x = mt_table,
        name = "count",
        marker=dict(
        color='#EB89B5'
    ),
      ),
    ],
    'layout': {
        'xaxis': {'title': 'Movie Type'},
        'yaxis': {'title': "User attention", 'type': 'log'}
    }
}

py.iplot(fig, filename='mt_overall_voting ')

/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [8]:
trace = go.Pie(labels=mt_table, values=mt_overall_voting)
py.iplot([trace],filename='mt_overall_voting ')

/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

